In [4]:
import aiohttp
from understat import Understat
import pandas as pd

In [ ]:
season_results = pd.read_csv("src/patenko/data/BAR_RMA_2023_understat/season_results.csv")

# Get the match IDs for previous matches of clubs before an El Clasico
real_prev_matches = season_results[((season_results['h.title'] == 'Real Madrid') | (season_results['a.title'] == 'Real Madrid')) & (season_results['datetime'] < '2023-10-28')]['id'].tolist()
barcelona_prev_matches = season_results[((season_results['h.title'] == 'Barcelona') | (season_results['a.title'] == 'Barcelona')) & (season_results['datetime'] < '2023-10-28')]['id'].tolist()
teams_prev_matches = real_prev_matches + barcelona_prev_matches

In [53]:
teams_shots = pd.DataFrame(columns = ['match_id', 'minute', 'result', 'xG', 'player', 'RB_team', 'against_team'])
against_shots = pd.DataFrame(columns = ['match_id', 'minute', 'result', 'xG', 'player', 'RB_team', 'against_team'])

# Asynchronously fetch match shots data using the Understat API
async with aiohttp.ClientSession() as session:
  understat = Understat(session)
  # Iterate through each match ID in the list of previous matches for teams
  for match_id in teams_prev_matches:
    # Fetch shots data for the current match
    shots = await understat.get_match_shots(match_id)
    # Iterate through each team's shots in the match
    for teams in shots.values():
      # Iterate through each player's shots in the team
      for players in teams:
        # Check if the player is from one of the specified teams (Real Madrid or Barcelona)
        if (players['h_a'] == 'h') and (players['h_team'] in ('Real Madrid', 'Barcelona')) or (players['h_a'] == 'a') and (players['a_team'] in ('Real Madrid', 'Barcelona')):
           # Add a row to the teams_shots DataFrame for shots by the specified team
           teams_shots.loc[len(teams_shots)] = [players['match_id'],
                                                players['minute'],
                                                players['result'],
                                                players['xG'],
                                                players['player'],
                                                players['h_team'] if players['h_a'] == 'h' else players['a_team'],
                                                players['a_team'] if players['h_a'] == 'h' else players['h_team']]
        else:
          # Add a row to the against_shots DataFrame for shots against the specified team
          against_shots.loc[len(against_shots)] = [players['match_id'],
                                                       players['minute'],
                                                       players['result'],
                                                       players['xG'],
                                                       players['player'],
                                                       players['a_team'] if players['h_a'] == 'h' else players['h_team'],
                                                       players['h_team'] if players['h_a'] == 'h' else players['a_team']]

In [56]:
teams_shots.to_csv("src/kondakov/data/understat")
against_shots.to_csv("src/kondakov/data/understat")